# Find Chemical Potential that Produces Some Average Gamma

The linker length distribution is highly dependent on the binding state of the system. The binding state of the system is highly dependent on the chemical potential. Search for a chemical potential that produces a linker length distribution with a desired average gamma value.

#### Specify Package Root Directory

In [1]:
import os
import sys

# Get the absolute path of the notebook's directory
notebook_directory = os.path.dirname(os.path.abspath('__file__'))

# Navigate one level up to get the root directory path
root_directory = os.path.abspath(os.path.join(notebook_directory, '..'))

# Change the working directory to the root directory
os.chdir(root_directory)

# Append the root directory to sys.path
sys.path.append(root_directory)

#### Import Modules

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sliding_nucleosome.nucleo_arr as nuc
from sliding_nucleosome import mc

#### Initialize Physical Parameters

In [3]:
# Specify physical parameters
J = np.atleast_2d([-3.92])
B = np.atleast_2d([-1.5])
mu = np.array([-9.7])

In [4]:
# Generate a methylation sequence
n_beads = 500
frac_methyl = 0.1
methyl_corr_length = 18.4
marks = nuc.gen_meth(n_beads, frac_methyl, methyl_corr_length)
marks = np.atleast_2d(marks).T

In [5]:
# Let's force the mark profile to match the one that should produce phase separation
marks = np.array(
    [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 2., 1., 1., 1., 1., 1., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 2., 1., 1., 1., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
   1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 2., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0.]
)
marks = np.atleast_2d(marks).T

In [6]:
# Specify the polymer
gamma = np.ones(marks.shape[0])
nbi = np.array([2])
linker_corr_length = 45
linker_lengths = np.random.exponential(linker_corr_length, size=marks.shape[0])
linker_lengths = np.maximum(linker_lengths, 1.0)
linker_lengths = linker_lengths.astype(int)
a = int(np.floor(15.1))
lam = -np.log(1 - 1 / linker_corr_length)

In [7]:
print(f"Linker Lengths: \n\n{linker_lengths}")

Linker Lengths: 

[  2  26   1  12  27  12  24  87 121  22  17  21 118  20  32  14  32  88
   1  29  25  13  16  12  80  45 170   1  15  80   6  25  51  34  71  48
  14  73   4  65  35  71  19  16  13   2 110  81   2  30   1   6  10  54
  82  42  66   2   2  15  28  33  32 139  47  66 116  13  97  16  70  13
 399 155  12 152  10  15  24  32  26  10  37  31  10   3   8   6   7  63
   8  12  17  13  13  43 162  28  19 307  43  12  90  59  24   8  65  13
  20  80   4  13  37  18 210   3  20  44   7  58  25   8  97  71   2  59
   4  27   3 174  14  56  67  19 153 103 117  68  33  38   1 110  63   7
 135  31  20  28  15  74  40   3   9  29  21   5  53  55  22  47   6  15
  16  23 108  65   3   5   6  22  22  12  19   6   9  96  62  23  16   8
   7  23 152  38   9   8  62 121 130  26  39   6  57  44  36  21  69   8
  29  81   1 149  27  56 140  12   6  51  90   9  60  23  37  24   4  59
  18  12  15  30  31  14  72  14  34  34  47  14   3  60   4  50  76  70
  34   8  56   1 102 173 112  27 

#### Initialize the Nucleosome Array

In [8]:
nuc_arr = nuc.NucleosomeArray(
    J = J,
    B = B,
    mu = mu,
    linker_lengths = linker_lengths,
    a = a,
    lam = lam,
    marks = marks,
    Nbi = nbi
)

In [9]:
print(f"Average Gamma: {np.average(nuc_arr.gamma)}")

Average Gamma: 0.332


#### Specify Parameters for Linker Simulation

In [10]:
n_snap = 1
n_steps_per_snap = 1000

#### Iterate Between Binding and Linker Simulations

Iterate in the following manner:

1. Run the binding model to select the chemical potential that produces the desired binding fraction for a fixed linker length distribution.

2. Using the chemical potential selected in step 1, update the linker length distribution using the linker model.

3. Iterate between steps 1 and 2 until the chemical potential and average `gamma` parameter converge.

In [11]:
target_avg_gamma = 0.5
mu_lower = -10.
mu_upper = -9.

mu = mc.find_mu_for_avg_gamma(
    nuc_arr,
    mu_lower,
    mu_upper,
    target_avg_gamma,
    n_snap,
    n_steps_per_snap
)

Starting iteration 2 of 10...
Starting iteration 4 of 10...
Starting iteration 6 of 10...
Starting iteration 8 of 10...
Starting iteration 10 of 10...
Maximum number of iterations have been met.
